In [16]:
import modin.pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objs as go

import plotly as pl

from datetime import datetime

In [3]:
train_df = pd.read_parquet('/home/alex/Projects/DataSci/ML/Kaggle/StoreSales/Datalake/train.parquet')
transactions_df = pd.read_parquet('/home/alex/Projects/DataSci/ML/Kaggle/StoreSales/Datalake/transactions.parquet')
stores_df = pd.read_parquet('/home/alex/Projects/DataSci/ML/Kaggle/StoreSales/Datalake/stores.parquet')
holidays_df = pd.read_parquet('/home/alex/Projects/DataSci/ML/Kaggle/StoreSales/Datalake/holidays_events.parquet')
oil_df = pd.read_parquet('/home/alex/Projects/DataSci/ML/Kaggle/StoreSales/Datalake/oil.parquet')


In [4]:
print(train_df['date'].min())
print(train_df['date'].max())

2013-01-01
2017-08-15


In [5]:
train_df.nlargest(20, 'sales')

,id,date,store_nbr,family,sales,onpromotion
2163723,2163723,2016-05-02,2,GROCERY I,124717.000,59
2445984,2445984,2016-10-07,39,MEATS,89576.360,0
2144154,2144154,2016-04-21,20,GROCERY I,87438.516,53
2139699,2139699,2016-04-18,45,GROCERY I,76090.000,38
2153031,2153031,2016-04-26,2,GROCERY I,63434.000,30
2145045,2145045,2016-04-21,45,GROCERY I,53874.000,44
562596,562596,2013-11-12,44,GROCERY I,46271.000,0
1257246,1257246,2014-12-08,35,GROCERY I,45361.000,3
1916586,1916586,2015-12-14,35,GROCERY I,40351.460,7
2760297,2760297,2017-04-02,9,GROCERY I,38422.625,73


In [6]:
# check data quality of train_df
train_df.isnull().sum()


id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
dtype: int64

In [7]:

store_sales = train_df.merge(stores_df, on='store_nbr', how='left')

In [8]:
# do a sum of sales grouped by cluster and date
cluster_sales = store_sales.groupby(['cluster', 'date']).agg({'sales': 'sum'}).reset_index()

In [9]:

cluster_sales['date'] = pd.to_datetime(cluster_sales['date'])
cluster_sales['week'] = cluster_sales['date'].dt.isocalendar().week
cluster_sales['year'] = cluster_sales['date'].dt.year
cluster_sales['year-week'] = cluster_sales['year'].astype(str) + '-' + cluster_sales['week'].astype(str)



Please refer to https://modin.readthedocs.io/en/stable/supported_apis/defaulting_to_pandas.html for explanation.


In [10]:

clusters = cluster_sales['cluster'].unique()

In [11]:
cluster_sales.head()

,cluster,date,sales,week,year,year-week
0,1,2013-01-01,2511.618999,1,2013,2013-1
1,1,2013-01-02,23542.623996,1,2013,2013-1
2,1,2013-01-03,18916.720018,1,2013,2013-1
3,1,2013-01-04,19490.300999,1,2013,2013-1
4,1,2013-01-05,24581.042040,1,2013,2013-1


In [12]:
weekly_sales = cluster_sales.groupby(['cluster', 'year-week']).agg({'sales': 'sum'}).reset_index()
weekly_sales = weekly_sales.sort_values(by=['cluster', 'year-week'])

In [17]:

cluster_1 = weekly_sales[weekly_sales['cluster'] == 1]
cluster_1['year-week-datetime'] = cluster_1['year-week'].apply(lambda x: datetime.strptime(x + '-1', '%Y-%W-%w'))
cluster_1 = cluster_1.sort_values(by='year-week-datetime')

fig = go.Figure()

fig.add_trace(go.Scatter(x=cluster_1['year-week'], 
                         y=cluster_1['sales'], 
                         mode='lines+markers', 
                         name='Cluster 1'))

fig.update_layout(title='Sales by Week for Cluster 1',
                  xaxis_title='Year-Week',
                  yaxis_title='Sales',
                  xaxis=dict(type='category'))  # Ensure x-axis is treated as categorical

fig.show()


In [18]:
fig = go.Figure()

clusters = weekly_sales['cluster'].unique()

for cluster in clusters:
    cluster_data = weekly_sales[weekly_sales['cluster'] == cluster]
    cluster_data['year-week-datetime'] = cluster_data['year-week'].apply(lambda x: datetime.strptime(x + '-1', '%Y-%W-%w'))
    cluster_data = cluster_data.sort_values(by='year-week-datetime')
    fig.add_trace(go.Scatter(x=cluster_data['year-week'], 
                             y=cluster_data['sales'], 
                             mode='lines+markers', 
                             name=f'{cluster}'))
    
fig.update_layout(title='Sales by Week for Clusters',
                  xaxis_title='Year-Week',
                  yaxis_title='Sales',
                  xaxis=dict(type='category'))  
fig.show()


In [1]:
# Ensure 'year-week' is sorted properly as a string in 'YYYY-WW' format
weekly_sales = weekly_sales.sort_values(by='')

# Create the figure
fig = go.Figure()

# Get the unique clusters
clusters = weekly_sales['cluster'].unique()

# Loop over each cluster and add a trace for each
for cluster in clusters:
    cluster_data = weekly_sales[weekly_sales['cluster'] == cluster]
    
    # Sort by 'year-week' before plotting
    cluster_data = cluster_data.sort_values(by='year-week')
    
    fig.add_trace(go.Scatter(x=cluster_data['year-week'], 
                             y=cluster_data['sales'], 
                             mode='lines+markers', 
                             name=f'Cluster {cluster}'))

# Update the layout
fig.update_layout(title='Sales by Week for Clusters',
                  xaxis_title='Year-Week',
                  yaxis_title='Sales',
                  xaxis=dict(type='category'))  # Keep 'year-week' as a category for proper display

# Show the plot
fig.show()

NameError: name 'weekly_sales' is not defined